In [1]:
from os import listdir
from os.path import isfile, join
import pandas as pd
from datetime import datetime

In [2]:
class Psi:
    def __init__(self, path=r'C:\Users\JGreenw9\JNJ\External Operations Plan - PSI Reports'):
        self.path = path
        self.files = self.get_files(self.path)
        self.filepaths = self.get_filepaths(self.path, self.files)

    def get_files(self, path):
        return [f for f in listdir(path) if isfile(join(path, f)) and '.xl' in f]

    def get_filepaths(self, path, files):
        return [path + '\\'+ x for x in files]

    def read_file(self, filepath):
        df = pd.DataFrame()
        if '.xlsx' in filepath:
            df = pd.read_excel(path)
        elif '.xlsb' in filepath:
            df = pd.read_excel(path, engine='pyxlsb')
        else:
            pd.DataFrame()
        df = self.init_prep_df(df)
        return df

    def get_date_header_conv(self, df):
        excel_dates = []
        for x in df.columns:
            try:
                int(x)
                excel_dates.append(x)
            except ValueError:
                pass
        dates = {}
        for date in excel_dates:
            dt = datetime.fromordinal(datetime(1900, 1, 1).toordinal() + int(date) - 2)
            result = str(dt.date())
            dates[date] = result
        return dates

    def init_prep_df(self, df : pd.DataFrame):
        df.columns = df.iloc[0]
        df = df.iloc[1:]
        renames = self.get_date_header_conv(df)
        df = df.rename(columns=renames)
        return df

    def get_oh(self, df):
        df = df[(df['Key Figure'] == 'Total Demand') | (df['Key Figure'] == 'Projected Inventory')]
        df = onhand[['LocationId', 'ProductId', '2014-01-01']].groupby(by=['LocationId', 'ProductId']).sum().reset_index()
        df['Key Figure'] = 'On Hand Inventory'
        df['Date'] = '2014-01-01'
        df = df.rename(columns={'2014-01-01': 'value'})
        return df

    def get_supply(self, df):
        df = df
        df = df[df['VendorDesc'].str.contains('-')]
        df = df[df['Key Figure'].isin(['Firm PO Receipt', 'Planned Order Receipt', 'Stock Movement'])]
        df = pd.melt(df, ['LocationId', 'ProductId', 'Key Figure'], list(dates.values()), var_name='Date')
        df = supply[df['value'] != 0].reset_index(drop=True)
        return df

    def combine_dfs(self, supply, onhand):
        df = pd.concat([onhand, supply]).reset_index(drop=True)
        df = pd.pivot(supply, ['LocationId', 'ProductId', 'Date'], 'Key Figure', 'value').reset_index()
        cal = pd.read_csv(r'\\na.jnj.com\dpyusdfsroot\RY_Company\Supply Chain Mgmt\Spine Plan-NPI\Teligen\Alteryx Deployment\Inputs\JNJ Calendar.csv')
        df = df.merge(cal[['Date (YYYY-MM-DD)', 'YYYYMM']],left_on=['Date'],right_on='Date (YYYY-MM-DD)').drop('Date (YYYY-MM-DD)', axis=1)
        df = df.sort_values(['ProductId', 'Date', 'LocationId'])
        return df

    def final_df(self, df):
        supply = self.get_supply(df)
        onhand = self.get_oh(df)
        df = self.combine_dfs(supply, onhand)
        return df

    def all_files(self, filepaths):
        dfs = []
        for filepath in filepaths:
            df = self.read_file(filepath)
            df = self.final_df(df)
            dfs.append(df)
        df = pd.concat(dfs)
        return df


In [4]:
path = r"\\na.jnj.com\dpyusdfsroot\RY_Company\Supply Chain Mgmt\Spine Plan-NPI\Conduit\Jonny\PSI\PSI 22-06-13 BW.xlsb"

In [5]:
psi = Psi()
print(psi.filepaths[1])
psi.read_file(psi.filepaths[1])

C:\Users\JGreenw9\JNJ\External Operations Plan - PSI Reports\PSI 22-06-13 LL.xlsx


In [35]:
raw_df = pd.read_excel(path, engine='pyxlsb')

In [275]:
df = raw_df.copy(deep=True)

In [276]:
df.columns = df.iloc[0]
df = df.iloc[1:]

In [277]:
excel_dates = []
for x in df.columns:
    try:
        int(x)
        excel_dates.append(x)
    except ValueError:
        pass

In [278]:
dates = {}
for date in excel_dates:
    dt = datetime.fromordinal(datetime(1900, 1, 1).toordinal() + int(date) - 2)
    tt = dt.timetuple()
    result = str(dt.date())
    dates[date] = result

In [279]:
df = df.rename(columns=dates)

In [280]:
onhand = df
onhand = onhand[(onhand['Key Figure'] == 'Total Demand') | (onhand['Key Figure'] == 'Projected Inventory')]

In [281]:
onhand = onhand[['LocationId', 'ProductId', '2014-01-01']].groupby(by=['LocationId', 'ProductId']).sum().reset_index()

In [282]:
onhand['Key Figure'] = 'On Hand Inventory'
onhand['Date'] = '2014-01-01'
onhand = onhand.rename(columns={'2014-01-01': 'value'})

In [283]:
onhand

,LocationId,ProductId,value,Key Figure,Date
0,CH20,108812007,29.0,On Hand Inventory,2014-01-01
1,CH20,108812008,100.0,On Hand Inventory,2014-01-01
2,CH20,108812009,109.0,On Hand Inventory,2014-01-01
3,CH20,108812010,114.0,On Hand Inventory,2014-01-01
4,CH20,108812011,75.0,On Hand Inventory,2014-01-01
...,...,...,...,...,...
14224,MORS,TY20004,7.0,On Hand Inventory,2014-01-01
14225,MORS,TY20005,11.0,On Hand Inventory,2014-01-01
14226,MORS,UT101001,7.0,On Hand Inventory,2014-01-01
14227,MORS,WB101007,19.0,On Hand Inventory,2014-01-01


In [284]:
df

,Profile,LocationId,VendorDesc,P4,ProductId,Description,MOQ,GRPT (Wks),Fixed Horizon,StockInRouting,...,2024-04-01,2024-04-08,2024-04-15,2024-04-22,2024-04-29,2024-05-06,2024-05-13,2024-05-20,2024-05-27,2024-06-03
1,ExOps Bridgewater 05/DS_SPF3/HUB,CH20,Transfer from MORS,PROTI 360 TPAL,108812007,"T-PAL PROTI, 10X7X28 MM",0,0,3w 3d,0,...,0.035714,14.035714,0.035714,8.035714,1.036714,23.035714,0.035714,9.035714,0.038095,0.038095
2,ExOps Bridgewater 05/DS_SPF3/HUB,CH20,Transfer from MORS,PROTI 360 TPAL,108812007,"T-PAL PROTI, 10X7X28 MM",0,0,3w 3d,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,ExOps Bridgewater 05/DS_SPF3/HUB,CH20,Transfer from MORS,PROTI 360 TPAL,108812007,"T-PAL PROTI, 10X7X28 MM",0,0,3w 3d,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,ExOps Bridgewater 05/DS_SPF3/HUB,CH20,Transfer from MORS,PROTI 360 TPAL,108812007,"T-PAL PROTI, 10X7X28 MM",0,0,3w 3d,0,...,0.000000,0.000000,0.000000,33.000000,0.000000,0.000000,0.000000,31.000000,0.000000,0.000000
5,ExOps Bridgewater 05/DS_SPF3/HUB,CH20,Transfer from MORS,PROTI 360 TPAL,108812007,"T-PAL PROTI, 10X7X28 MM",0,0,3w 3d,0,...,22.000000,0.000000,0.000000,0.000000,33.000000,0.000000,0.000000,0.000000,31.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156515,ExOps Bridgewater 02/DS_SPY1/HUB,MORS,0430039751 - Tecomet,REMOVAL KIT,WB101009,"WRENCH, OPEN END",5,2,30w 0d,0,...,51.577956,51.422514,51.267073,51.111632,50.956191,50.800749,50.645308,50.489867,50.335971,50.182076
156516,ExOps Bridgewater 02/DS_SPY1/HUB,MORS,0430039751 - Tecomet,REMOVAL KIT,WB101009,"WRENCH, OPEN END",5,2,30w 0d,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
156517,ExOps Bridgewater 02/DS_SPY1/HUB,MORS,0430039751 - Tecomet,REMOVAL KIT,WB101009,"WRENCH, OPEN END",5,2,30w 0d,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
156518,ExOps Bridgewater 02/DS_SPY1/HUB,MORS,0430039751 - Tecomet,REMOVAL KIT,WB101009,"WRENCH, OPEN END",5,2,30w 0d,0,...,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000


In [285]:
supply = df
supply = supply[supply['VendorDesc'].str.contains('-')]

In [286]:
supply = supply[supply['Key Figure'].isin(['Firm PO Receipt', 'Planned Order Receipt', 'Stock Movement'])]

In [287]:
supply = pd.melt(supply, ['LocationId', 'ProductId', 'Key Figure'], list(dates.values()), var_name='Date')

In [288]:
supply = supply[supply['value'] != 0].reset_index(drop=True)

In [289]:
supply = pd.concat([onhand, supply]).reset_index(drop=True)

In [290]:
supply = pd.concat([onhand, supply]).reset_index(drop=True)
supply = pd.pivot(supply, ['LocationId', 'ProductId', 'Date'], 'Key Figure', 'value').reset_index()
cal = pd.read_csv(r'\\na.jnj.com\dpyusdfsroot\RY_Company\Supply Chain Mgmt\Spine Plan-NPI\Teligen\Alteryx Deployment\Inputs\JNJ Calendar.csv')
supply = supply.merge(cal[['Date (YYYY-MM-DD)', 'YYYYMM']],left_on=['Date'],right_on='Date (YYYY-MM-DD)').drop('Date (YYYY-MM-DD)', axis=1)
supply = supply.sort_values(['ProductId', 'Date', 'LocationId'])

In [291]:
cal = pd.read_csv(r'\\na.jnj.com\dpyusdfsroot\RY_Company\Supply Chain Mgmt\Spine Plan-NPI\Teligen\Alteryx Deployment\Inputs\JNJ Calendar.csv')

In [292]:
cal.columns

Index(['Date (YYYY-MM-DD)', 'Date (DD/MM/YYYY)', 'JNJ Year (YYYY)',
       'No Day in JNJ year (D)', 'No Day in JNJ year (YYYYD)', 'JNJ Week (W)',
       'JNJ Month (M)', 'JNJ Week (YYYY/WW)', 'JNJ Start of Week (DD/MM/YYYY)',
       'JNJ Month & Year (01/MM/YYYY)', 'JNJ Month Start Date (DD/MM/YYYY)',
       'JNJ Year Start Date (DD/MM/YYYY)', 'Public Holiday Name',
       'Public Holiday Flag', 'Working Day', 'Quarter (Q)', 'Month in Quarter',
       'Date (DDD)', 'Date (D)', 'PivotFormatting', 'JNJ Month (MM - MMM)',
       'JNJ Quarter (QQ - "QRT")', 'YYYYMM'],
      dtype='object')

In [293]:
supply = supply.merge(cal[['Date (YYYY-MM-DD)', 'YYYYMM']],left_on=['Date'],right_on='Date (YYYY-MM-DD)').drop('Date (YYYY-MM-DD)', axis=1)
supply = supply.sort_values(['ProductId', 'Date', 'LocationId'])

In [295]:
supply.to_clipboard(index=False)